## BLERSSI Pipeline Deployment in UCS

### Clone git repo

In [1]:
BRANCH_NAME="dev" #Provide git branch "master" or "dev"
! git clone -b $BRANCH_NAME https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

Cloning into 'cisco-kubeflow-starter-pack'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 4604 (delta 9), reused 21 (delta 5), pack-reused 4567
Receiving objects: 100% (4604/4604), 17.60 MiB | 4.76 MiB/s, done.
Resolving deltas: 100% (1738/1738), done.


In [2]:
!pip install kfp --user

     |████████████████████████████████| 122kB 559kB/s eta 0:00:01
     |████████████████████████████████| 61kB 4.1MB/s eta 0:00:011
     |████████████████████████████████| 92kB 4.0MB/s eta 0:00:011
  Created wheel for kfp: filename=kfp-0.5.1-cp36-none-any.whl size=163152 sha256=b3745656098a7e3d692554721da69e8a0b6df8b15a0809f9e876b5769fba7129
  Stored in directory: /home/jovyan/.cache/pip/wheels/39/d0/75/dc86a70b812b917c357015a37fe780afe806dd8403ed83365f
  Created wheel for kfp-server-api: filename=kfp_server_api-0.5.0-cp36-none-any.whl size=106324 sha256=d5cbe25d8afd1334cd23d1bcd73f2c3648ccb2210d7c8b27dec1e5d14dcfada7
  Stored in directory: /home/jovyan/.cache/pip/wheels/d0/85/34/cdf006df2be7a462c85d8760ffea2db4e65c3c6efeb3f2f4ec
  Created wheel for strip-hints: filename=strip_hints-0.1.9-py2.py3-none-any.whl size=24671 sha256=10b64c77d3176a95893a1f9b2166abd72bb3da46a5a133e6ff99fe180638e729
  Stored in directory: /home/jovyan/.cache/pip/wheels/1a/6c/b0/2e38ee5cf60250da7f47962f8ab888672

## Restart Notebook Kernel

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

### Import libraries

In [1]:
import kfp
import os
from kubernetes import client
import calendar
import time

### Component files Declarations

In [2]:
path='cisco-kubeflow-starter-pack/apps/networking/ble-localization/onprem/pipelines/'
component_root_katib= path+'components/v2/tf-katib/'
component_root_train= path+'components/v2/tf-model-train/'
component_root_serve = path+'components/v2/tf-serving-deploy-service/'
component_root_webui = path+'components/v2/tf-webui-deploy-service/'
component_root_tensorboard= path+'components/v2/tf-tensorboard-deploy-service/'

#### Components Description

tf_katib_op &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - &nbsp;&nbsp;&nbsp; Run katib for BLERSSI to get optimal hyperparamater values and use it to train model. <br>
tf_train_model_op &nbsp;&nbsp;&nbsp; - &nbsp; Load dataset from nfs-volume, train BLERSSI model  and save model in nfs-volume. <br> 
tf_serve_model_op &nbsp; - &nbsp; Load BLERSSI model from nfs-volume and serve the model using tf-serving for later prediction. <br> 
tf_webui_op &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - &nbsp; Web application client UI loads Unlabeled data, does preprocessing, send API request to serving and display Predicted Location. <br> 
tf_tensor_op &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  - &nbsp; Tensorboard UI which provides the visualization metrics such as loss and accuracy, model graph (ops and layers) and histograms of weights, biases, or other tensors for BLERSSI Model <br> 

### Load cpu or gpu component yaml

In [3]:
processor = "CPU" # Provide GPU or CPU as variable to include which processor to use
if processor == "GPU" : component_file="component_gpu.yaml"
else : component_file="component_cpu.yaml"
    
processor

'CPU'

### Load components from respective .YAML config files

In [4]:
tf_katib_op = kfp.components.load_component_from_file(os.path.join(component_root_katib, 'component.yaml')) 
tf_train_model_op = kfp.components.load_component_from_file(os.path.join(component_root_train, component_file)) 
tf_serve_model_op = kfp.components.load_component_from_file(os.path.join(component_root_serve, 'component.yaml'))
tf_webui_op = kfp.components.load_component_from_file(os.path.join(component_root_webui, 'component.yaml'))
tf_tensor_op = kfp.components.load_component_from_file(os.path.join(component_root_tensorboard, 'component.yaml'))

### Define Volume and Volume Mounts

In [5]:
nfs_pvc = client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs')
nfs_volume = client.V1Volume(name='nfs', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = client.V1VolumeMount(mount_path='/mnt/', name='nfs')

### Define pipeline function

In [6]:
# Creating timestamp
timestamp = str(calendar.timegm(time.gmtime()))
print(timestamp)
def blerssi_pipeline():

    #Defining Task to run katib
    tf_katib_task = tf_katib_op(timestamp=timestamp, image="docker.io/poornimadevii/blerssi-train:v1")
    tf_katib_task.add_volume(nfs_volume)
    tf_katib_task.add_volume_mount(nfs_volume_mount)    
    
        
    #Defining Task for Model training
    tf_train_model_task = tf_train_model_op(timestamp=timestamp)
    tf_train_model_task.add_volume(nfs_volume)
    tf_train_model_task.add_volume_mount(nfs_volume_mount)
    tf_train_model_task.after(tf_katib_task)
    
    #Defining Task to perform serving    
    tf_serve_model_task = tf_serve_model_op(timestamp=timestamp)
    tf_serve_model_task.add_volume(nfs_volume)
    tf_serve_model_task.add_volume_mount(nfs_volume_mount)
    tf_serve_model_task.after(tf_train_model_task)

    #Defining task for tensorboard creation    
    tf_tensorboard_task = tf_tensor_op(timestamp=timestamp)
    tf_tensorboard_task.add_volume(nfs_volume)
    tf_tensorboard_task.add_volume_mount(nfs_volume_mount)
    tf_tensorboard_task.after(tf_train_model_task)
    
    
    #Defining task for webUI creation
    tf_webui_task = tf_webui_op(image_path="docker.io/premkarthi/blerssi-webui:v4",
                                timestamp=timestamp)
    tf_webui_task.add_volume(nfs_volume)
    tf_webui_task.add_volume_mount(nfs_volume_mount)
    tf_webui_task.after(tf_serve_model_task)
    
#Creating a pipeline run
kfp.Client().create_run_from_pipeline_func(blerssi_pipeline, arguments={})

1594719378


RunPipelineResult(run_id=79568535-09ee-4901-9aaa-22316d5a1634)

### Before accessing service URLs below, make sure that Pipeline Run is Complete in the Dashboard

### Get Tensorboard URL

In [7]:
TB_SERVICE_NAME = "tensorboard-"+str(timestamp)+"-blerssi-service"
port = !(kubectl get svc {TB_SERVICE_NAME} -n kubeflow -o yaml | grep -i nodePort: | cut -d":" -f 2)
print('Tensorboard service is available at - http://INGRESS_IP:{}'.format(port[0].lstrip()))

Tensorboard service is available at - http://INGRESS_IP:30888


### Get WebUI URL 
And upload csv from https://github.com/CiscoAI/cisco-kubeflow-starter-pack/blob/master/apps/networking/ble-localization/onprem/data/iBeacon_RSSI_Unlabeled_truncated.csv

In [8]:
WEBUI_SERVICE_NAME = "webui-"+str(timestamp)+"-blerssi-service"
port = !(kubectl get svc {WEBUI_SERVICE_NAME} -n kubeflow -o yaml | grep -i nodePort: | cut -d":" -f 2)
print('WebUI service is available at - http://INGRESS_IP:{}'.format(port[0].lstrip()))

WebUI service is available at - http://INGRESS_IP:31270
